In [2]:
import pandas as pd
import datetime as dt
from datetime import datetime, timedelta
from suntime import Sun, SunTimeException
from datetime import datetime, timezone
import numpy as np


def time_in_range(start, end, x):
    """Return true if x is in the range [start, end]"""
    if start <= end:
        return start <= x <= end
    else:
        return start <= x or x <= end   

def timesteps_light_juelich(startdate, enddate, path, name):
    '''
    What does it?
    -> In a timeperiod (between startdate and enddate) creates a list with 5 minutes timesteps, 
    during daylight (one hour after sunrise and one hour after sunrise),
    Saves it as a dataset, where Year, Month and Day are datavariables
    
    input:
        - startdate: <string>: e.g.: '2020-01-01' 
        - enddate:   <string>: e.g.: '2020-03-02'
        - path:      <string>: where to save the nc-file eg.: 'savings/'
        - name:      <string>: name of the nc-file: 'daylight.nc'
        
    ''' 
    #coordinates of Juelich
    latitude = 50.9224
    longitude = 6.3639
    sun = Sun(latitude, longitude)

    #period of time you want tocheck for daylight
    all_dates = pd.date_range(start=startdate,end=enddate)

    light_flag = []
    date_string = []
    hour_string = []
    minute_string = []
    t_all = []

    for day in all_dates:

        #getting the time of sunrise +1 hour and the time of sunset - 1 hour. (result has information of timezone)
        sur_tzd = sun.get_local_sunrise_time(day) + timedelta(hours=1)
        sus_tzd = sun.get_local_sunset_time(day) - timedelta(hours=1)

        #delete timezonedesignator (otherwise it can't sompare if a certain time is within the range later (func time_in_range))
        sur = sur_tzd.replace(tzinfo=None)
        sus = sus_tzd.replace(tzinfo=None)

        #create an array with all timestamps (every five minutes) for the day
        next_day = day + timedelta(days=1)
        t = np.arange(day,next_day, timedelta(minutes=5)).astype(datetime)

        # check if the timestamps are between the sunrise and sunset
        for moment in t: 
            # check if the timestamps are between the sunrise and sunset
            start = sur
            end = sus
            light_flag.append(time_in_range(start, end, moment))

            #convert timstamps in strings (yyyymmdd, hh, mm)
            date_string.append(moment.strftime("%Y") + moment.strftime("%m") + moment.strftime("%d"))
            hour_string.append(moment.strftime('%H'))
            minute_string.append(moment.strftime('%M'))

        #create a list with all days
        t_all = np.concatenate((t_all, t))

    #create a dataframe
    d = {'date': date_string, 'hour': hour_string, 'minute': minute_string,  'light_flag': light_flag}
    df = pd.DataFrame(data=d)

    df.drop(df[df['light_flag'] == False].index, inplace = True)
    df = df.drop(columns=['light_flag'])

    #create dataset and save it as an nc-File
    ds = df.to_xarray()
    path_and_name = path + name
    ds.to_netcdf(path_and_name)  


In [ ]:
import pandas as pd
import datetime as dt
from datetime import datetime, timedelta
from suntime import Sun, SunTimeException
from datetime import datetime, timezone
import numpy as np
import glob
import os
import xarray as xr
from pathlib import Path

def timesteps_light_juelich(startdate, enddate, path):
    '''
    What does it?
    -> In a timeperiod (between startdate and enddate) opens the daily datasets and selects the timstamps with sunlight.
    Saves it as datasets in a given path. It creates new folders for every year if they do not already exist.
    e.g.: 'path/2018/sups_joy_pyrg00_l1_rlds_v01_daylight20180102000000.nc'
    
    input:
        - startdate: <string>: e.g.: '2020-01-01' 
        - enddate:   <string>: e.g.: '2020-03-02'
        - path:      <string>: where to save the nc-file eg.: 'savings/'

        
    ''' 

    #Range of the datas
    all_dates = pd.date_range(start=startdate,end=enddate)

    #coordinates of Juelich
    latitude = 50.908546
    longitude = 6.413536
    sun = Sun(latitude, longitude)

    for day in all_dates:
        
        
        #getting the time of sunrise +1 hour and the time of sunset - 1 hour. (result has information of timezone)
        sur_tzd = sun.get_local_sunrise_time(day) + timedelta(hours=1)
        sus_tzd = sun.get_local_sunset_time(day) - timedelta(hours=1)

        #delete timezonedesignator (Data from sup_joys are in UTC)
        sur = sur_tzd.replace(tzinfo=None)
        sus = sus_tzd.replace(tzinfo=None)
        #print(sur, 'sunrise')
        

        
        #get the day to string, to open the right file
        y = day.strftime('%Y')
        m = day.strftime('%m')
        d = day.strftime('%d')
        date = y + m + d + '000000.nc'

        #Open Files for LW and shortwave data and put them in one dataset
        
        #starting with LW
        #getting the names from the data. Some names have ..._v00_... and some ..._v01...
        fileName_LW = glob.glob('/data/hatpro/jue/hdcp2/radiation_hdcp2/'+ y +'/sups_joy_pyrg00_l1_rlds_v*_'+ date )
        #if there is no file for the day it goes to the next day
        if not fileName_LW:
            print(date + ' no LW-file for this day found')
            continue
            
        #open files 
        fileObj_LW = Path(str(fileName_LW[0]))
        if fileObj_LW.is_file() == True:
            ds_LW = xr.open_dataset(str(fileName_LW[0]))
            
            #in a really few cases it didn't work out to select times (I guess something is wrong with the datasets). Nevertheless that the program doesent stop there is a try:...
            try:
                ds_LW_light = ds_LW.sel(time=slice(sur, sus))
            
            except:
                print(str(fileName_LW[0]),' something went wrong with selecting sunlight time')
           
            #checking if path/folder already exists
            MYDIR = (path + y)
            CHECK_FOLDER = os.path.isdir(MYDIR)

            # If folder doesn't exist, then create it.
            if not CHECK_FOLDER:
                os.makedirs(MYDIR)
                print("created folder : ", MYDIR)
                
            #saving it as a new dataset 
            save_LW = path + y + '/sups_joy_pyrg00_l1_rlds_v_daylight' + date
            ds_LW_light.to_netcdf(save_LW)  

        else: 
            print('File not found')
            print(fileName[0])

        #SW
        #getting the names from the data. Some names have ..._v00_... and some ..._v01...
        fileName_SW = glob.glob('/data/hatpro/jue/hdcp2/radiation_hdcp2/'+ y +'/sups_joy_pyr00_l1_rsds_v*_'+ date )
        #if there is no file for the day it goes to the next day
        if not fileName_SW:
            print(date + ' no SW-file for this day found')
            continue
            
        #open files 
        fileObj_SW = Path(str(fileName_SW[0]))
        if fileObj_LW.is_file() == True:
            ds_SW = xr.open_dataset(str(fileName_SW[0]))
            
            #in a really few cases it didn't work out to select times (I guess something is wrong with the datasets). Nevertheless that the program doesent stop there is a try:...
            try:
                ds_SW_light = ds_SW.sel(time=slice(sur, sus))
            
            except:
                print(str(fileName_LW[0]),' something went wrong with selecting sunlight time')
           
            #checking if path/folder already exists
            MYDIR = (path + y)
            CHECK_FOLDER = os.path.isdir(MYDIR)

            # If folder doesn't exist, then create it.
            if not CHECK_FOLDER:
                os.makedirs(MYDIR)
                print("created folder : ", MYDIR)
                
            #saving it as a new dataset 
            save_SW = path + y + '/sups_joy_pyr00_l1_rsds_v_daylight' + date
            ds_SW_light.to_netcdf(save_SW)  

        else: 
            print('File not found')
            print(fileName[0]) 



timesteps_light_juelich('2011-01-01', '2022-06-01', 'savings/')


20110101000000.nc no LW-file for this day found
20110102000000.nc no LW-file for this day found
20110103000000.nc no LW-file for this day found
20110104000000.nc no LW-file for this day found
20110105000000.nc no LW-file for this day found
20110106000000.nc no LW-file for this day found
20110107000000.nc no LW-file for this day found
20110108000000.nc no LW-file for this day found
20110109000000.nc no LW-file for this day found
20110110000000.nc no LW-file for this day found
20110111000000.nc no LW-file for this day found
20110112000000.nc no LW-file for this day found
20110113000000.nc no LW-file for this day found
20110114000000.nc no LW-file for this day found
20110115000000.nc no LW-file for this day found
20110116000000.nc no LW-file for this day found
20110117000000.nc no LW-file for this day found
20110118000000.nc no LW-file for this day found
20110119000000.nc no LW-file for this day found
20110120000000.nc no LW-file for this day found
20110121000000.nc no LW-file for this da

20110628000000.nc no LW-file for this day found
20110629000000.nc no LW-file for this day found
20110630000000.nc no LW-file for this day found
20110701000000.nc no LW-file for this day found
20110702000000.nc no LW-file for this day found
20110703000000.nc no LW-file for this day found
20110704000000.nc no LW-file for this day found
20110705000000.nc no LW-file for this day found
20110706000000.nc no LW-file for this day found
20110707000000.nc no LW-file for this day found
20110708000000.nc no LW-file for this day found
20110709000000.nc no LW-file for this day found
20110710000000.nc no LW-file for this day found
20110711000000.nc no LW-file for this day found
20110712000000.nc no LW-file for this day found
20110713000000.nc no LW-file for this day found
20110714000000.nc no LW-file for this day found
20110715000000.nc no LW-file for this day found
20110716000000.nc no LW-file for this day found
20110717000000.nc no LW-file for this day found
20110718000000.nc no LW-file for this da

20111224000000.nc no SW-file for this day found
20111225000000.nc no SW-file for this day found
20111226000000.nc no SW-file for this day found
20111227000000.nc no SW-file for this day found
20111228000000.nc no SW-file for this day found
20111229000000.nc no SW-file for this day found
20111230000000.nc no SW-file for this day found
20111231000000.nc no SW-file for this day found
created folder :  savings/2012
20120101000000.nc no SW-file for this day found
20120102000000.nc no SW-file for this day found
20120103000000.nc no SW-file for this day found
20120104000000.nc no SW-file for this day found
20120105000000.nc no SW-file for this day found
20120106000000.nc no SW-file for this day found
20120107000000.nc no SW-file for this day found
20120108000000.nc no SW-file for this day found
20120109000000.nc no SW-file for this day found
20120110000000.nc no SW-file for this day found
20120111000000.nc no SW-file for this day found
20120112000000.nc no SW-file for this day found
201201130

20120612000000.nc no SW-file for this day found
20120613000000.nc no SW-file for this day found
20120614000000.nc no SW-file for this day found
20120615000000.nc no SW-file for this day found
20120616000000.nc no SW-file for this day found
20120617000000.nc no SW-file for this day found
20120618000000.nc no SW-file for this day found
20120619000000.nc no SW-file for this day found
20120620000000.nc no SW-file for this day found
20120621000000.nc no SW-file for this day found
20120622000000.nc no SW-file for this day found
20120623000000.nc no SW-file for this day found
20120624000000.nc no SW-file for this day found
20120625000000.nc no SW-file for this day found
20120626000000.nc no SW-file for this day found
20120627000000.nc no SW-file for this day found
20120628000000.nc no SW-file for this day found
20120629000000.nc no SW-file for this day found
20120630000000.nc no SW-file for this day found
20120701000000.nc no SW-file for this day found
20120702000000.nc no SW-file for this da

20130502000000.nc no SW-file for this day found
20130503000000.nc no SW-file for this day found
20130504000000.nc no SW-file for this day found
20130505000000.nc no SW-file for this day found
20130506000000.nc no SW-file for this day found
20130507000000.nc no SW-file for this day found
20130508000000.nc no SW-file for this day found
20130509000000.nc no SW-file for this day found
20130510000000.nc no SW-file for this day found
20130511000000.nc no SW-file for this day found
20130512000000.nc no SW-file for this day found
20130513000000.nc no SW-file for this day found
20130514000000.nc no SW-file for this day found
20130515000000.nc no SW-file for this day found
20130516000000.nc no SW-file for this day found
20130517000000.nc no SW-file for this day found
20130518000000.nc no SW-file for this day found
20130519000000.nc no SW-file for this day found
20130520000000.nc no SW-file for this day found
20130521000000.nc no SW-file for this day found
20130522000000.nc no SW-file for this da

20131025000000.nc no SW-file for this day found
20131026000000.nc no SW-file for this day found
20131027000000.nc no SW-file for this day found
20131028000000.nc no SW-file for this day found
20131029000000.nc no SW-file for this day found
20131030000000.nc no SW-file for this day found
20131031000000.nc no SW-file for this day found
20131101000000.nc no SW-file for this day found
20131102000000.nc no SW-file for this day found
20131103000000.nc no SW-file for this day found
20131104000000.nc no SW-file for this day found
20131105000000.nc no SW-file for this day found
20131106000000.nc no SW-file for this day found
20131107000000.nc no SW-file for this day found
20131108000000.nc no SW-file for this day found
20131109000000.nc no SW-file for this day found
20131110000000.nc no SW-file for this day found
20131111000000.nc no SW-file for this day found
20131112000000.nc no SW-file for this day found
20131113000000.nc no SW-file for this day found
20131114000000.nc no SW-file for this da

20170501000000.nc no SW-file for this day found
20170502000000.nc no SW-file for this day found
20170503000000.nc no SW-file for this day found
20170504000000.nc no SW-file for this day found
20170505000000.nc no SW-file for this day found
20170506000000.nc no SW-file for this day found
20170507000000.nc no SW-file for this day found
20170508000000.nc no SW-file for this day found
20170509000000.nc no SW-file for this day found
20170510000000.nc no SW-file for this day found
20170511000000.nc no SW-file for this day found
20170512000000.nc no SW-file for this day found
20170513000000.nc no SW-file for this day found
20170514000000.nc no SW-file for this day found
20170515000000.nc no SW-file for this day found
20170516000000.nc no SW-file for this day found
20170517000000.nc no SW-file for this day found
20170518000000.nc no SW-file for this day found
20170519000000.nc no SW-file for this day found
20170520000000.nc no SW-file for this day found
20170521000000.nc no SW-file for this da

20171019000000.nc no SW-file for this day found
20171020000000.nc no SW-file for this day found
20171021000000.nc no SW-file for this day found
20171022000000.nc no SW-file for this day found
20171023000000.nc no SW-file for this day found
20171024000000.nc no SW-file for this day found
20171025000000.nc no SW-file for this day found
20171026000000.nc no SW-file for this day found
20171027000000.nc no SW-file for this day found
20171028000000.nc no SW-file for this day found
20171029000000.nc no SW-file for this day found
20171030000000.nc no SW-file for this day found
20171031000000.nc no SW-file for this day found
20171101000000.nc no SW-file for this day found
20171102000000.nc no SW-file for this day found
20171103000000.nc no SW-file for this day found
20171104000000.nc no SW-file for this day found
20171105000000.nc no SW-file for this day found
20171106000000.nc no SW-file for this day found
20171107000000.nc no SW-file for this day found
20171108000000.nc no SW-file for this da

20180411000000.nc no SW-file for this day found
20180412000000.nc no SW-file for this day found
20180413000000.nc no SW-file for this day found
20180414000000.nc no SW-file for this day found
20180415000000.nc no SW-file for this day found
20180416000000.nc no SW-file for this day found
20180417000000.nc no SW-file for this day found
20180418000000.nc no SW-file for this day found
20180614000000.nc no LW-file for this day found
20180615000000.nc no LW-file for this day found
20180616000000.nc no LW-file for this day found
20180617000000.nc no LW-file for this day found
20180618000000.nc no LW-file for this day found
created folder :  savings/2019
20190506000000.nc no LW-file for this day found
created folder :  savings/2020
20200214000000.nc no LW-file for this day found


In [33]:
import pandas as pd
import datetime as dt
from datetime import datetime, timedelta
from suntime import Sun, SunTimeException
from datetime import datetime, timezone
import numpy as np

import os
import pandas as pd
import xarray as xr
from pathlib import Path

#Range of the datas
all_dates = pd.date_range(start='2017-01-25',end='2017-01-26')

#Dä Plän: Datasets für Tage öffnen, dann direkt Zeitraum für Sonnenlicht auswählen und dann an vorherigen Tag dranhängen

#coordinates of Juelich
latitude = 50.908546
longitude = 6.413536
sun = Sun(latitude, longitude)

for day in all_dates:
    
    #get the day to string, to open the right file
    y = day.strftime('%Y')
    m = day.strftime('%m')
    d = day.strftime('%d')
    date = y + m + d + '000000.nc'
    
    #Open Files for LW and shortwave data and put them in one dataset
    #LW-Data
    fileName = '/data/hatpro/jue/hdcp2/radiation_hdcp2/'+ y +'/sups_joy_pyrg00_l1_rlds_v01_'+ date
    fileObj = Path(fileName)
    if fileObj.is_file() == True:
        ds_lw = xr.open_dataset(fileName)
    else: 
        print('File not found')
        print(fileName)
        
    #SW-Data    
    fileName = '/data/hatpro/jue/hdcp2/radiation_hdcp2/'+ y +'/sups_joy_pyr02_l1_rsds_v00_'+ date
    fileObj = Path(fileName)
    if fileObj.is_file() == True:
        ds_sw = xr.open_dataset(fileName)
    else: 
        print('File not found')
        print(fileName)

    #put the sw data to the lw data
    ds_lw['rsds'] = ds_sw['rsds']
    ds_lw['rsds_error'] = ds_sw['rsds_error']
    ds = ds_lw
    
        
    #getting the time of sunrise +1 hour and the time of sunset - 1 hour. (result has information of timezone)
    sur_tzd = sun.get_local_sunrise_time(day) + timedelta(hours=1)
    sus_tzd = sun.get_local_sunset_time(day) - timedelta(hours=1)
    
    #delete timezonedesignator (Data from sup_joys are in UTC)
    sur = sur_tzd.replace(tzinfo=None)
    sus = sus_tzd.replace(tzinfo=None)
    
    ds_light_one_day = ds.sel(time=slice(sur, sus))
    ds_light_one_day









#create dataset and save it as an nc-File
#ds = df.to_xarray()
#path_and_name = path + name
#ds.to_netcdf(path_and_name)  


File not found
/data/hatpro/jue/hdcp2/radiation_hdcp2/2017/sups_joy_pyrg00_l1_rlds_v01_20170125000000.nc
File not found
/data/hatpro/jue/hdcp2/radiation_hdcp2/2017/sups_joy_pyrg00_l1_rlds_v01_20170126000000.nc


In [38]:
#SW-Data    
fileName = '/data/hatpro/jue/hdcp2/radiation_hdcp2/2017/sups_joy_pyrg02_l1_rlds_v01_20170122000000.nc'
fileObj = Path(fileName)
if fileObj.is_file() == True:
    ds_sw = xr.open_dataset(fileName)
    ds_sw_l = ds_sw.sel(time=slice('2017-01-22 09:26:00', '2017-01-22 10:00:05'))
else: 
    print('File not found')
    print(fileName)
        
ds_sw

<xarray.Dataset>
Dimensions:     (time: 17280, nv: 2)
Coordinates:
  * time        (time) datetime64[ns] 2017-02-22 ... 2017-02-22T23:59:55
Dimensions without coordinates: nv
Data variables:
    time_bnds   (time, nv) datetime64[ns] ...
    zsl         int16 ...
    lat         float32 ...
    lon         float32 ...
    rlds        (time) float32 ...
    rlds_error  (time) float32 ...
Attributes:
    Title:            Downwelling long wave irradiance
    Institution:      Research Center Juelich, Institute for Energy and Clima...
    Contact_person:   Birger Bohn (b.bohn@fz-juelich.de)
    Source:           Kipp & Zonen pyrgeometer SGR4
    History:          Data processed by IEK-8 and written with radiation2nc_h...
    Conventions:      CF-1.6 where applicable
    Processing_date:  2018-04-25 15:48:56 (CET)
    Author:           Lukas Pfitzenmaier (l.pfitzenmaier@uni-koeln.de)
    Comments:          
    Licence:          For non-commercial use only. This data is subject to th...
    Dependencies:     external

In [13]:
import os
import pandas as pd
import xarray as xr
from pathlib import Path

all_dates = pd.date_range(start='2018-01-01',end='2018-01-01')
print(all_dates)
for day in all_dates:
    y = day.strftime('%Y')
    m = day.strftime('%m')
    d = day.strftime('%d')
    date = y + m + d + '000000.nc'
    fileName = '/data/hatpro/jue/hdcp2/radiation_hdcp2/'+ y +'/sups_joy_pyrg00_l1_rlds_v01_'+ date
    fileObj = Path(fileName)
    if fileObj.is_file() == True:
        ds = xr.open_dataset(fileName)
    else: 
        print('File not found')
        print(fileName)

    #os.chdir('/data/hatpro/jue/hdcp2/radiation_hdcp2/'+y+'/ups_joy_pyrg00_l1_rlds_v01_'+ date)
ds

DatetimeIndex(['2018-01-01'], dtype='datetime64[ns]', freq='D')
01


<xarray.Dataset>
Dimensions:     (time: 17280, nv: 2)
Coordinates:
  * time        (time) datetime64[ns] 2018-01-01T00:00:03 ... 2018-01-01T23:5...
Dimensions without coordinates: nv
Data variables:
    time_bnds   (time, nv) datetime64[ns] ...
    zsl         int16 ...
    lat         float32 ...
    lon         float32 ...
    rlds        (time) float32 ...
    rlds_error  (time) float32 ...
Attributes:
    Title:            Downwelling long wave irradiance
    Institution:      Research Center Juelich, Institute for Energy and Clima...
    Contact_person:   Birger Bohn (b.bohn@fz-juelich.de)
    Source:           Kipp & Zonen pyrgeometer CGR4
    History:          Data processed by IEK-8 and written with do_radiation2n...
    Conventions:      CF-1.6 where applicable
    Processing_date:  2018-11-23 11:43:07 (CET)
    Author:           Lukas Pfitzenmaier (l.pfitzenmaier@uni-koeln.de)
    Comments:         none
    Licence:          For non-commercial use only. This data is subject to th...
    Dependencies:     external

In [53]:
infile = '/data/obs/site/jue/pvm/l1/2016/10/details20161005.nc.gz'

import gzip
import shutil
with gzip.open('/data/obs/site/jue/pvm/l1/2016/10/details20161005.nc.gz', 'rb') as f_in:
    with open('unzipped_data/details20161005.nc', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
        
fileName = 'details20161005.nc'
fileObj = Path(fileName)
if fileObj.is_file() == True:
    ds = xr.open_dataset(fileName)
else: 
    print('File not found')
    print(fileName)      
ds

<xarray.Dataset>
Dimensions:        (device_number: 2, time10s: 3444, time60s: 574)
Coordinates:
  * device_number  (device_number) uint8 1 2
  * time10s        (time10s) datetime64[ns] 2016-10-05T14:26:00 ... 2016-10-0...
  * time60s        (time60s) datetime64[ns] 2016-10-05T14:26:00 ... 2016-10-0...
Data variables:
    serial_number  (device_number) object ...
    ocvv           (time10s, device_number) float32 ...
    sccv           (time10s, device_number) float32 ...
    mppvv          (time10s, device_number) float32 ...
    mppcv          (time10s, device_number) float32 ...
    irrv           (time10s) uint64 ...
    mtv            (time10s, device_number) float32 ...
    stv            (time10s, device_number) float32 ...
    hst            (time60s, device_number) float32 ...
    tem            (time60s, device_number) float32 ...
    status         (time60s, device_number) uint8 ...
    ivchar         (time60s, device_number) object ...
Attributes:
    desciption:  This nc file contains measurements of two photovoltaic modul...
    history:     Created: 2017-07-13 12:48:11.654760
    source:      /data/obs/site/jue/pvm/l1/2016/10/details20161005.txt.gz

TypeError: a bytes-like object is required, not 'str'

In [34]:


import gzip
import os
import shutil
import tempfile

import netCDF4

def open_netcdf(fname):
    if fname.endswith(".gz"):
        infile = gzip.open(fname, 'rb')
        tmp = tempfile.NamedTemporaryFile(delete=False)
        shutil.copyfileobj(infile, tmp)
        infile.close()
        tmp.close()
        data = netCDF4.Dataset(tmp.name)
        os.unlink(tmp.name)
    else:
        data = netCDF4.Dataset(fname)
    return data

open_netcdf('/data/obs/site/jue/pvm/l1/2016/10/details20161005.nc.gz')

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    desciption: This nc file contains measurements of two photovoltaic modules placed at JOYCE. Some of the values have a 10 second time resolution, some have a 60 second time resolution. The file therefore got two time variables: time10s for the ten second resolution and time60s for the sixty second resolution
    history: Created: 2017-07-13 12:48:11.654760
    source: /data/obs/site/jue/pvm/l1/2016/10/details20161005.txt.gz
    dimensions(sizes): device_number(2), time10s(3444), time60s(574)
    variables(dimensions): uint8 device_number(device_number), float64 time10s(time10s), float64 time60s(time60s), <class 'str'> serial_number(device_number), float32 ocvv(time10s, device_number), float32 sccv(time10s, device_number), float32 mppvv(time10s, device_number), float32 mppcv(time10s, device_number), uint64 irrv(time10s), float32 mtv(time10s, device_number), float32 stv(time10s, device_number), float

In [46]:
import os
os.chdir('/data/hatpro/jue/hdcp2/radiation_hdcp2/2018')
os.listdir()

['sups_joy_pyrg00_l1_rlds_v01_20180419000000.nc',
 'sups_joy_pyrg00_l1_rlds_v01_20181201000000.nc',
 'sups_joy_pyrg00_l1_rlds_v01_20180420000000.nc',
 'sups_joy_pyr00_l1_rsds_v01_20181201000000.nc',
 'sups_joy_pyrg00_l1_rlds_v01_20180421000000.nc',
 'sups_joy_pyrg00_l1_rlds_v01_20181202000000.nc',
 'sups_joy_pyrg00_l1_rlds_v01_20180422000000.nc',
 'sups_joy_pyr00_l1_rsds_v01_20181202000000.nc',
 'sups_joy_pyrg00_l1_rlds_v01_20180423000000.nc',
 'sups_joy_pyrg00_l1_rlds_v01_20181203000000.nc',
 'sups_joy_pyrg00_l1_rlds_v01_20180424000000.nc',
 'sups_joy_pyr00_l1_rsds_v01_20181203000000.nc',
 'sups_joy_pyrg00_l1_rlds_v01_20180425000000.nc',
 'sups_joy_pyrg00_l1_rlds_v01_20181204000000.nc',
 'sups_joy_pyrg00_l1_rlds_v01_20180426000000.nc',
 'sups_joy_pyr00_l1_rsds_v01_20181204000000.nc',
 'sups_joy_pyrg00_l1_rlds_v01_20180427000000.nc',
 'sups_joy_pyrg00_l1_rlds_v01_20181205000000.nc',
 'sups_joy_pyrg00_l1_rlds_v01_20180428000000.nc',
 'sups_joy_pyr00_l1_rsds_v01_20181205000000.nc',
 'sup

In [4]:
#how to open the dataset and read it to use it for the program

import xarray as xr
from pathlib import Path

startdate = '2020-01-01'
enddate = '2020-02-01'
path = 'savings/'
name = 'daylight'

#call function
timesteps_light_juelich(startdate, enddate, path, name)


fileName = path + name
fileObj = Path(fileName)
if fileObj.is_file() == True:
    ds = xr.open_dataset(fileName)
else: 
    print('File not found') 

dates = ds.date.data
hours = ds.hour.data
minutes = ds.minute.data

print(dates)
print(hours)
print(minutes)

['20200101' '20200101' '20200101' ... '20200201' '20200201' '20200201']
['09' '09' '09' ... '16' '16' '16']
['40' '45' '50' ... '15' '20' '25']
